In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
import re
import string

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.exceptions import UndefinedMetricWarning

In [73]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from Levenshtein import distance as levenshtein_distance

In [4]:
import json
import random
import warnings
from datetime import datetime

In [5]:
from tqdm import tqdm

In [6]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

#### Visualisation

In [7]:
import matplotlib
import matplotlib.pyplot as plt

In [8]:
import scienceplots

plt.style.use('science')
%config InlineBackend.figure_format = 'retina'

lables_fs = 16
ticks_fs = 12

## Prepare `train` and `test` datasets from data

In [118]:
prepared_dir = '../data/prepared'
filename_csv = '01_punct_pushkin.csv'

In [119]:
# load saved dataset
dataset_df = pd.read_csv(os.path.join(prepared_dir, filename_csv), index_col=0)
dataset_df.shape

(4456, 4)

In [120]:
dataset_df = dataset_df.drop('input_lemma', axis=1)
dataset_df = dataset_df.drop('input_pos', axis=1)

In [121]:
PUNC_2_TOKEN = {' ': 'S', ',': 'C', '.': 'P', '!': 'EX', '?': 'Q'}
TOKEN_2_PUNC = {v: k for k, v in PUNC_2_TOKEN.items()}

In [122]:
output_targets = []

for index, row in tqdm(dataset_df.iterrows(), total=dataset_df.shape[0]):
    sent_with_punc = ''
    for word, token in zip(row['input'].split(), row['target'].split()):
        sent_with_punc = sent_with_punc + word + TOKEN_2_PUNC[token]

    output_targets.append(sent_with_punc)

dataset_df['output_target'] = output_targets

100%|████████████████████████████████████| 4456/4456 [00:00<00:00, 27831.07it/s]


In [123]:
pd.options.display.max_colwidth = 150
dataset_df.sample(5)

,input,target,output_target
2960,церковь полна была кистеневскими крестьянами пришедшими отдать последнее поклонение господину своему,S S S S C S S S S S P,"церковь полна была кистеневскими крестьянами,пришедшими отдать последнее поклонение господину своему."
4364,муромский провозгласивший себя отличным наездником дал ей волю и внутренне доволен был случаем избавлявшим его от неприятного собеседника,C S S S C S S S S S S S C S S S S P,"муромский,провозгласивший себя отличным наездником,дал ей волю и внутренне доволен был случаем,избавлявшим его от неприятного собеседника."
10,доложили что мусье давал мне свой урок,C S S S S S P,"доложили,что мусье давал мне свой урок."
3752,мы простились еще раз и лошади поскакали,S S S C S S P,"мы простились еще раз,и лошади поскакали."
1996,начали очищать ров окружающий город а вал обносить рогатками,S S C S C S S S P,"начали очищать ров,окружающий город,а вал обносить рогатками."


## Model from Hugging Face

We will use [the token classification model](https://huggingface.co/markusiko/rubert-base-punctuation) from Hugging Face for russian punctuation.

In [112]:
from transformers import pipeline, AutoModelForTokenClassification

In [113]:
model_path = 'markusiko/rubert-base-punctuation'

model_pretrained = AutoModelForTokenClassification.from_pretrained(model_path)

In [17]:
punct_pipeline = pipeline('token-classification', model=model_pretrained)

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [67]:
sent = 'в тридцати шагах промаху в карту не дам разумеется из знакомых пистолетов'

sent_punc_preds = punct_pipeline(sent)  # prediction
sent_punc_preds

[{'entity': 'B-,',
  'score': 0.94763273,
  'index': 9,
  'word': 'дам',
  'start': 36,
  'end': 39},
 {'entity': 'B-,',
  'score': 0.98577046,
  'index': 10,
  'word': 'разумеется',
  'start': 40,
  'end': 50},
 {'entity': 'B-.',
  'score': 0.9234911,
  'index': 13,
  'word': 'пистолетов',
  'start': 63,
  'end': 73}]

In [132]:
# make the output readable + in target form
ALL_PUNC = '.,:;!?'
UNK_TOKEN = 'U'


def get_sentence_with_punctuation(sent, model_preds, all_entities=[]):

    sent_words = sent.split()
    sent_new = (sent + ' ')[:-1]
    
    n_marks = 0
    for pred in model_preds:
        if pred['entity'] not in all_entities:
            all_entities.append(pred['entity'])

        punc_mark = pred['entity'][-1]

        ind_to_place = pred['end'] + n_marks
        if len(sent_new) == ind_to_place:  # last mark
            sent_new = sent_new + punc_mark
        else:
            if (sent_new[ind_to_place] == ' '):
                sent_new = sent_new[:ind_to_place] + punc_mark + sent_new[ind_to_place:]
                n_marks += 1

    target_like = []
    for word in sent_new.split():
        last_chr = word[-1]
        if last_chr in ALL_PUNC:
            if last_chr in PUNC_2_TOKEN.keys():
                target_like.append(PUNC_2_TOKEN[last_chr])
            else:
                target_like.append(UNK_TOKEN)
        else:
            target_like.append(PUNC_2_TOKEN[' '])

    return sent_new, ' '.join(target_like), all_entities

In [133]:
print(f'Original    : {sent}')
print(f'Processed   : {get_sentence_with_punctuation(sent, sent_punc_preds)[0]}')
print(f'Target-like : {get_sentence_with_punctuation(sent, sent_punc_preds)[1]}')

Original    : в тридцати шагах промаху в карту не дам разумеется из знакомых пистолетов
Processed   : в тридцати шагах промаху в карту не дам, разумеется, из знакомых пистолетов.
Target-like : S S S S S S S C C S S P


### Predict all data

In [134]:
model_preds = []
model_preds_as_target = []

all_entities = []

for index, row in tqdm(dataset_df.iterrows(), total=dataset_df.shape[0]):
    sent_with_punc, target_like, all_entities = get_sentence_with_punctuation(
        row['input'], punct_pipeline(row['input']), all_entities
    )
    model_preds.append(sent_with_punc)
    model_preds_as_target.append(target_like)

100%|███████████████████████████████████████| 4456/4456 [03:51<00:00, 19.22it/s]


In [136]:
all_entities  # all entities of the model

['B-,', 'B-.', 'B-?', 'B-!', 'B-...']

In [124]:
dataset_df['output'] = model_preds
dataset_df['output_punc'] = model_preds_as_target

In [125]:
dataset_df.sample(5)

,input,target,output_target,output,output_punc
3851,через полчаса маша должна была навсегда оставить родительский дом свою комнату тихую девическую жизнь,S S S S S S S S C S C S S P,"через полчаса маша должна была навсегда оставить родительский дом,свою комнату,тихую девическую жизнь.","через полчаса маша должна была навсегда оставить родительский дом, свою комнату, тихую, девическую жизнь.",S S S S S S S S C S C C S P
1432,меня привезли в крепость уцелевшую посереди сгоревшего города,S S S C S S S P,"меня привезли в крепость,уцелевшую посереди сгоревшего города.","меня привезли в крепость, уцелевшую посереди сгоревшего города.",S S S C S S S P
2566,преследовать его было невозможно у михельсона не было и тридцати годных лошадей,S S S C S S S S S S S P,"преследовать его было невозможно,у михельсона не было и тридцати годных лошадей.",преследовать его было невозможно. у михельсона не было и тридцати годных лошадей.,S S S P S S S S S S S P
2325,пугачев выступил против князя голицына с десятью тысячами отборного войска оставя под оренбургом шигаева с двумя тысячами,S S S S S S S S S C S S S S S S P,"пугачев выступил против князя голицына с десятью тысячами отборного войска,оставя под оренбургом шигаева с двумя тысячами.","пугачев выступил против князя голицына с десятью тысячами отборного войска, оставя под оренбургом шигаева с двумя тысячами.",S S S S S S S S S C S S S S S S P
2545,никто не знал что уже накануне михельсон в семи верстах от города имел жаркое дело с пугачевым и что мятежники отступили в беспорядке,S S C S S S S S S S S S S S S S S S S S S S P,"никто не знал,что уже накануне михельсон в семи верстах от города имел жаркое дело с пугачевым и что мятежники отступили в беспорядке.","никто не знал, что уже накануне михельсон в семи верстах от города имел жаркое дело с пугачевым и что мятежники отступили в беспорядке.",S S C S S S S S S S S S S S S S S S S S S S P


### Metrics _from the box_

In [149]:
END_PUNC = ['P', 'EX', 'Q']
INTR_PUNC = ['S', 'C']

NAMES_PUNC = {
    'S': 'space (` `)',
    'C': 'comma (`,`)',
    'P': 'point (`.`)',
    'EX': 'excl. (`!`)',
    'Q': 'question (`?`)',
    'U': 'other'
}

CLASSES = sorted(END_PUNC + INTR_PUNC)  # alphabetic order

In [150]:
def return_separate_punct(target_vs_pred_df, pred_col_name='output_punc'):
    test_all_punc_target = []  # list of all punctuation
    test_all_punc_preds = []
    
    for target_this, predicted_this in zip(target_vs_pred_df['target'], target_vs_pred_df[pred_col_name]):
        test_all_punc_target.extend(target_this.split(' '))
        test_all_punc_preds.extend(predicted_this.split(' '))
    
    assert len(test_all_punc_target) == len(test_all_punc_preds)
    
    return test_all_punc_target, test_all_punc_preds


def get_all_metrics(test_df, pred_col_name='output_punc'):
    test_all_punc_target, test_all_punc_preds = return_separate_punct(
        test_df, pred_col_name=pred_col_name
    )

    cm = confusion_matrix(test_all_punc_target, test_all_punc_preds)
    # precision = TP / (TP + FP)
    precision = precision_score(test_all_punc_target, test_all_punc_preds, average=None, zero_division=np.nan)
    # recall = TP / (TP + FN)
    recall = recall_score(test_all_punc_target, test_all_punc_preds, average=None, zero_division=np.nan)
    # f1 = 2TP / (2TP + FP + FN)
    f1 = f1_score(test_all_punc_target, test_all_punc_preds, average=None)

    # PRINT
    metrics_names = ['Precision', 'Recall', 'F1 score']
    metrics = {'Precision': precision, 'Recall': recall, 'F1 score': f1}
    col_w = 16
    
    print(' ' * col_w + '|' + ''.join([f"{NAMES_PUNC[token] + (col_w - len(NAMES_PUNC[token])) * ' '}|" for token in CLASSES]))  # header
    print(''.join(['-' * col_w + '|' for _ in range(len(CLASSES) + 1)]) )
    for ind, metric_name in enumerate(metrics_names):
        row = f"{metric_name + (col_w - len(metric_name)) * ' '}|"
        for score in metrics[metric_name]:
            score_str = f'{score:.6f}'
            row += f"{score_str + (col_w - len(score_str)) * ' '}|"
        print(row)

In [151]:
%%time
get_all_metrics(dataset_df)

                |comma (`,`)     |excl. (`!`)     |point (`.`)     |question (`?`)  |space (` `)     |
----------------|----------------|----------------|----------------|----------------|----------------|
Precision       |0.913819        |0.405941        |0.763220        |0.567123        |0.975520        |
Recall          |0.774770        |0.232955        |0.960040        |0.824701        |0.983370        |
F1 score        |0.838569        |0.296029        |0.850390        |0.672078        |0.979430        |
CPU times: user 362 ms, sys: 14.7 ms, total: 377 ms
Wall time: 379 ms


## Additional training on our data (fine-tuning)

### Dataset

In [227]:
from datasets import Dataset, DatasetDict
from datasets import Features, ClassLabel, Value, Sequence

In [255]:
TOKEN_2_ENTITY = {
    'S': 'O',
    'C': 'B-,',
    'P': 'B-.',
    'EX': 'B-!',
    'Q': 'B-?',
}

ENTITY_2_ID = {
    'O': 0,
    'B-,': 1,
    'B-.': 2,
    'B-!': 3,
    'B-?': 4,
    'B-:': 5,
    'B-...': 6
}
ID_2_ENTITY = {v: k for k, v in ENTITY_2_ID.items()}

In [256]:
splitting_random_state = 78
test_ratio = 0.25

train_ids, test_ids = train_test_split(
    np.arange(dataset_df.shape[0]),
    test_size=test_ratio,
    random_state=splitting_random_state
)

In [257]:
my_dataset = {'train': [], 'test': []}

for ds_part_name, ids in zip(('train', 'test'), (train_ids, test_ids)):
    print(ds_part_name, len(ids), sep=': ')
    
    for i, df_ind in enumerate(tqdm(ids)):
        item = {}
        item['id'] = i
        
        ner_tags = []
        for token in dataset_df.iloc[df_ind]['target'].split():
            ner_tags.append(ENTITY_2_ID[TOKEN_2_ENTITY[token]])
        item['ner_tags'] = ner_tags
        
        item['tokens'] = dataset_df.iloc[df_ind]['input'].split()
    
        my_dataset[ds_part_name].append(item)

train: 3342


100%|████████████████████████████████████| 3342/3342 [00:00<00:00, 15012.87it/s]


test: 1114


100%|████████████████████████████████████| 1114/1114 [00:00<00:00, 24204.84it/s]


In [258]:
my_features = Features(
    {
        'id': Value(dtype='int64', id=None),
        'ner_tags': Sequence(
            feature=ClassLabel(
                num_classes=len(ENTITY_2_ID),
                names=list(ENTITY_2_ID.keys()),
                names_file=None, id=None
            ), length=-1, id=None
        ),
        'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
    }
)

In [259]:
my_train_dataset = Dataset.from_list(my_dataset['train'], features=my_features)
my_test_dataset = Dataset.from_list(my_dataset['test'], features=my_features)

In [260]:
my_test_dataset[1001]

{'id': 1001,
 'ner_tags': [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2],
 'tokens': ['вдруг',
  'дубровский',
  'вздрогнул',
  'в',
  'укреплении',
  'сделалась',
  'тревога',
  'и',
  'степка',
  'просунул',
  'к',
  'нему',
  'голову',
  'в',
  'окошко']}

In [261]:
my_train_test_dataset = DatasetDict(
    {
    'train': my_train_dataset,
    'test': my_test_dataset,
    }
)

In [262]:
my_train_test_dataset['train'][7]

{'id': 7,
 'ner_tags': [1, 1, 0, 0, 0, 2],
 'tokens': ['оба', 'соединясь', 'поспешили', 'вслед', 'за', 'пугачевым']}

In [263]:
my_train_dataset.features

{'id': Value(dtype='int64', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-,', 'B-.', 'B-!', 'B-?', 'B-:', 'B-...'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [264]:
label_list = my_train_dataset.features['ner_tags'].feature.names
label_list

['O', 'B-,', 'B-.', 'B-!', 'B-?', 'B-:', 'B-...']

### From [tutorial](https://huggingface.co/docs/transformers/tasks/token_classification)

In [265]:
from transformers import AutoTokenizer

In [266]:
from transformers import DataCollatorForTokenClassification

In [267]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [268]:
example = my_train_test_dataset['train'][0]
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
tokens

['[CLS]', 'хлеба', 'ради', 'хри', '##ста', 'хлеба', '[SEP]']

In [269]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [270]:
tokenized_my_ds = my_train_test_dataset.map(
    tokenize_and_align_labels, batched=True
)

Map:   0%|          | 0/3342 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1114 [00:00<?, ? examples/s]

In [271]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

#### Evaluate

In [272]:
import evaluate

In [273]:
seqeval = evaluate.load("seqeval")

In [274]:
labels = [label_list[i] for i in example['ner_tags']]
labels

['B-,', 'O', 'B-,', 'B-.']

In [275]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

#### Train

In [288]:
import accelerate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [289]:
model_my = AutoModelForTokenClassification.from_pretrained(
    model_path, 
    num_labels=len(ID_2_ENTITY),
    id2label=ID_2_ENTITY,
    label2id=ENTITY_2_ID,
    ignore_mismatched_sizes=True
)

In [290]:
training_args = TrainingArguments(
    output_dir='my_finetuned_hf_punc',
    learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
)

trainer = Trainer(
    model=model_my,
    args=training_args,
    train_dataset=my_train_test_dataset['train'],
    eval_dataset=my_train_test_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`